In [1]:
import numpy as np
from collections import defaultdict as dd
from typing import List, Dict, Set
import nltk
from math import log, ceil
import time
import sys
import itertools
from numba import jit, njit, jitclass

In [2]:
import pickle
with open("Dane/wiki_list", "rb") as ifile:
    wiki_list = pickle.load(ifile)

In [3]:
wiki_list[549]

('Calvados (napój alkoholowy)',
 [['Calvados', '(', 'napój', 'alkoholowy', ')'],
  ['Calvados',
   '–',
   'wytrawny',
   'winiak',
   'francuski',
   ',',
   'charakteryzujący',
   'się',
   'wytrawnym',
   'smakiem',
   'owocowym',
   '(',
   'jabłkowym',
   ')',
   'i',
   'charakterystycznym',
   'jabłkowym',
   'bukietem',
   '.',
   'Wyrabiany',
   'jest',
   'głównie',
   'w',
   'departamencie',
   'Calvados',
   'w',
   'Normandii',
   'poprzez',
   'destylację',
   'cydru',
   '.'],
  ['Nazwa',
   '``',
   'calvados',
   '``',
   '(',
   'będąca',
   'pochodną',
   'nazwy',
   'departamentu',
   ')',
   ',',
   'podobnie',
   'jak',
   'nazwy',
   'koniak',
   'i',
   'armagnac',
   ',',
   'jest',
   'prawnie',
   'zastrzeżona',
   ',',
   'dostępna',
   'tylko',
   'dla',
   'produktów',
   'z',
   'Normandii',
   ',',
   'dlatego',
   'też',
   'produkowane',
   'np',
   '.',
   'w',
   'Polsce',
   'wypalanki',
   'jabłkowe',
   'noszą',
   'nazwy',
   'wódek',
   'w',
  

In [4]:
class TrieNode:
    def __init__(self):
        self.word = None
        self.index = None
        self.children = {}

    def insert( self, word, index = None):
        word = word.lower()
        node = self
        for letter in word:
            if letter not in node.children: 
                node.children[letter] = TrieNode()

            node = node.children[letter]

        node.word = word
        node.index = index

In [5]:
print(trie.children)

NameError: name 'trie' is not defined

In [ ]:
# read dictionary file into a trie
trie = TrieNode()
word_set = set()
title_mapping = dd(lambda:-1)
for i, el in enumerate(wiki_list):
    title_mapping[" ".join(el[1][0])] = i
#     for w in el[1][0]:
#         if not w.isalpha():
#             continue
#         w = w.lower()
#         word_set.add(w)
#         trie.insert(w, i)
    trie.insert(el[0], i)

In [ ]:
# The search function returns a list of all words that are less than the given
# maximum distance from the target word
# @jit
def search( word, maxCost ):

    # build first row
    currentRow = range( len(word) + 1 )

    results = []

    # recursively search each branch of the trie
    for letter in trie.children:
        searchRecursive( trie.children[letter], letter, word, currentRow, 
            results, maxCost )

    return results

In [ ]:
from multiprocessing import Pool

In [ ]:
_word = ""
_maxCost = 2
def _search(letter):
    global _word
    global _maxCost
    currentRow = range( len(_word) + 1 )
    results = []
    searchRecursive(trie.children[letter], letter, _word, currentRow, 
            results, _maxCost )
    return results
def search_parallel(word, maxCost):
    global _word
    global _maxCost
    _word = word
    _maxCost = maxCost
    p = Pool(100)
    res = p.map(_search, trie.children.keys())
    of_the_jedi = []
    for el in res:
        of_the_jedi += el
    return of_the_jedi

In [ ]:
# This recursive helper is used by the search function above. It assumes that
# the previousRow has been filled in already.
# @jit
def searchRecursive( node, letter, word, previousRow, results, maxCost ):

    columns = len( word ) + 1
    currentRow = [ previousRow[0] + 1 ]

    # Build one row for the letter, with a column for each letter in the target
    # word, plus one for the empty string at column 0
    for column in range( 1, columns ):

        insertCost = currentRow[column - 1] + 1
        deleteCost = previousRow[column] + 1

        if word[column - 1] != letter:
            replaceCost = previousRow[ column - 1 ] + 1
        else:                
            replaceCost = previousRow[ column - 1 ]

        currentRow.append( min( insertCost, deleteCost, replaceCost ) )

    # if the last entry in the row indicates the optimal cost is less than the
    # maximum cost, and there is a word in this trie node, then add it.
    if currentRow[-1] <= maxCost and node.word != None:
        results.append( (node.index, currentRow[-1] ) )
#         results.append( (node.word) )


    # if any entries in the row are less than the maximum cost, then 
    # recursively search each branch of the trie
    if min( currentRow ) <= maxCost:
        for letter in node.children:
            searchRecursive( node.children[letter], letter, word, currentRow, 
                results, maxCost )

In [ ]:
# @jit
def search_title(title):
    limit = 2
    title = title.lower()
    while limit <= 512:
        res = search(title, limit)
        if len(res) > 0:
            return [wiki_list[el[0]][0] for el in sorted(res, key = lambda el: el[1])]
        limit *= 2

In [ ]:
# @jit
def search_title_parallel(title):
    limit = 2
    title = title.lower()
    while limit <= 512:
        res = search_parallel(title, limit)
        if len(res) > 0:
            return [wiki_list[el[0]][0] for el in sorted(res, key = lambda el: el[1])]
        limit *= 2

In [ ]:
%%timeit
print(search_title("uniwersytet wroclawaski"))

In [ ]:
%%timeit
print(search_title_parallel("uniwersytet wroclawaski"))

In [ ]:
def search_title1(title):
    corrected = []
    for w in nltk.word_tokenize(title):
        w = w.lower()
        if not w.isalpha() or w in corrected:
            corrected.append([w])
            continue
        limit = 2
        prop = []
        while limit <= 8 and prop == []:
            prop = search(title, limit)
            limit *= 2
        corrected.append(prop)
    propositions = itertools.product(*corrected)
    print(propositions)
    of_the_jedi = [title_mapping[p] for p in propositions if title_mapping[p] >= 0]
    return of_the_jedi